In [ ]:
import os
import glob
import xml.etree.ElementTree as ET
import yaml
from PIL import Image
from ultralytics import YOLO

def convert_bbox_to_yolo(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x_center = (box[0] + box[2]) / 2.0
    y_center = (box[1] + box[3]) / 2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    
    x_center_norm = x_center * dw
    y_center_norm = y_center * dh
    w_norm = w * dw
    h_norm = h * dh
    
    return (x_center_norm, y_center_norm, w_norm, h_norm)

def process_dataset_split(dataset_root, split, class_to_idx):
    img_dir = os.path.join(dataset_root, "images", split)
    xml_dir = os.path.join(dataset_root, f"{split}_annots")
    label_dir = os.path.join(dataset_root, "labels", split)
    
    os.makedirs(label_dir, exist_ok=True)

    if not (os.path.exists(xml_dir) and os.path.exists(img_dir)):
        return

    xml_files = glob.glob(os.path.join(xml_dir, "*.xml"))
    img_files_in_split = {os.path.basename(f) for f in glob.glob(os.path.join(img_dir, "*"))}
    
    for xml_path in xml_files:
        base = os.path.splitext(os.path.basename(xml_path))[0]
        label_path = os.path.join(label_dir, f"{base}.txt")
        yolo_lines = []
        
        try:
            root = ET.parse(xml_path).getroot()
            objects_found = [obj for obj in root.findall('object') if obj.findtext('name') in class_to_idx]
            
            if not objects_found:
                with open(label_path, 'w') as f: f.write('')
                continue

            size_elem = root.find('size')
            img_w, img_h = 0, 0
            
            if size_elem is not None:
                img_w = int(size_elem.findtext('width'))
                img_h = int(size_elem.findtext('height'))
            else:
                img_path = None
                img_found = False
                for ext in ['.png','.jpg','.jpeg','.tif', '.tiff']:
                    potential_file = base + ext
                    if potential_file in img_files_in_split:
                        img_path = os.path.join(img_dir, potential_file)
                        img_found = True
                        break
                if not img_found: continue
                with Image.open(img_path) as img:
                    img_w, img_h = img.size
            
            for obj in objects_found:
                class_id = class_to_idx[obj.findtext('name')]
                bnd = obj.find('bndbox')
                if bnd is None: continue
                
                xmin = int(float(bnd.findtext('xmin')))
                ymin = int(float(bnd.findtext('ymin')))
                xmax = int(float(bnd.findtext('xmax')))
                ymax = int(float(bnd.findtext('ymax')))
                
                yolo_bbox = convert_bbox_to_yolo((img_w, img_h), (xmin, ymin, xmax, ymax))
                yolo_lines.append(f"{class_id} {' '.join(map(str, yolo_bbox))}")
            
            with open(label_path, 'w') as f: f.write('\n'.join(yolo_lines))
                
        except Exception as e:
            print(f"Error processing {xml_path}: {e}")

def prepare_dataset(dataset_root, dataset_name, classes_to_train):
    class_to_idx = {name: i for i, name in enumerate(classes_to_train)}
    
    yaml_data = {
        'path': dataset_root,
        'train': os.path.join('images', 'train'),
        'val': os.path.join('images', 'valid'),
        'test': os.path.join('images', 'test'),
        'names': classes_to_train
    }
    
    yaml_path = os.path.join(dataset_root, f'{dataset_name.replace(" ", "_").lower()}_dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_data, f, sort_keys=False, default_flow_style=False)
    
    for split in ["train", "valid", "test"]:
        process_dataset_split(dataset_root, split, class_to_idx)
        
    return yaml_path

DATASET_NAME = "UAV 1"
BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "Datasets"))
CLASSES_TO_TRAIN = ['building']
RUN_NAME = f'yolo11n_{DATASET_NAME.replace(" ", "_").lower()}_building_only_no_aug_clean'

EPOCHS = 100
BATCH_SIZE = 24
WORKERS = 12
IMGSZ = 640

dataset_root = os.path.join(BASE_PATH, DATASET_NAME)
yaml_config_path = os.path.join(dataset_root, f'{DATASET_NAME.replace(" ", "_").lower()}_dataset.yaml')
train_labels_path = os.path.join(dataset_root, "labels", "train")

if not os.path.exists(dataset_root):
    print(f"Error: Dataset root not found at {dataset_root}")
else:
    if not (os.path.exists(yaml_config_path) and os.path.exists(train_labels_path)):
        prepare_dataset(dataset_root, DATASET_NAME, CLASSES_TO_TRAIN)
    
    model = YOLO('yolo11n.pt') 
    try:
        model.train(
            data=yaml_config_path,
            epochs=EPOCHS,
            imgsz=IMGSZ,
            batch=BATCH_SIZE,
            workers=WORKERS,
            name=RUN_NAME,
            verbose=False,
            hsv_h=0.0, hsv_s=0.0, hsv_v=0.0,
            degrees=0.0, translate=0.0, scale=0.0, shear=0.0,
            perspective=0.0, flipud=0.0, fliplr=0.0, bgr=0.0,
            mosaic=0.0, mixup=0.0, copy_paste=0.0
        )

        best_model_path = os.path.join('runs', 'detect', RUN_NAME, 'weights', 'best.pt')
        if os.path.exists(best_model_path):
            model = YOLO(best_model_path)
            model.val(split='test', verbose=False)
        else:
            print(f"Validation failed: Could not find best model at {best_model_path}")
    
    except Exception as e:
        print(f"\nAn error occurred during training: {e}")


New https://pypi.org/project/ultralytics/8.3.223 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.40  Python-3.11.9 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=c:\Users\Think\Desktop\NesvarbuSefoSpecialus\Datasets\UAV 1\uav_1_dataset.yaml, epochs=100, time=None, patience=100, batch=24, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=12, project=None, name=yolo11n_uav_1_building_only_no_aug_clean5, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augm

train: Scanning C:\Users\Think\Desktop\NesvarbuSefoSpecialus\Datasets\UAV 1\labels\train.cache... 717 images, 181 backgrounds, 0 corrupt: 100%|██████████| 717/717 [00:00<?, ?it/s]
val: Scanning C:\Users\Think\Desktop\NesvarbuSefoSpecialus\Datasets\UAV 1\labels\valid.cache... 84 images, 21 backgrounds, 0 corrupt: 100%|██████████| 84/84 [00:00<?, ?it/s]
c:\Users\Think\anaconda3\envs\P170M109\Lib\site-packages\seaborn\_statistics.py:32: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.1)
  from scipy.stats import gaussian_kde


Plotting labels to runs\detect\yolo11n_uav_1_building_only_no_aug_clean5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005625000000000001), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 12 dataloader workers
Logging results to runs\detect\yolo11n_uav_1_building_only_no_aug_clean5
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      6.42G      2.076      2.747      1.197        200        640: 100%|██████████| 30/30 [00:10<00:00,  2.81it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.57it/s]

                   all         84       1391     0.0123      0.222     0.0502     0.0175


...


100 epochs completed in 0.176 hours.
Optimizer stripped from runs\detect\yolo11n_uav_1_building_only_no_aug_clean5\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\yolo11n_uav_1_building_only_no_aug_clean5\weights\best.pt, 5.5MB

Validating runs\detect\yolo11n_uav_1_building_only_no_aug_clean5\weights\best.pt...
Ultralytics 8.3.40  Python-3.11.9 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.25it/s]


                   all         84       1391       0.71      0.624      0.673      0.344
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\yolo11n_uav_1_building_only_no_aug_clean5
Ultralytics 8.3.40  Python-3.11.9 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients


val: Scanning C:\Users\Think\Desktop\NesvarbuSefoSpecialus\Datasets\UAV 1\labels\test.cache... 43 images, 11 backgrounds, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:04<00:00,  1.36s/it]


                   all         43        630      0.695      0.584      0.605      0.291
Speed: 1.5ms preprocess, 11.6ms inference, 0.0ms loss, 6.0ms postprocess per image
Results saved to runs\detect\val2


In [ ]:
import os
import glob
import xml.etree.ElementTree as ET
import yaml
from PIL import Image
from ultralytics import YOLO

def convert_bbox_to_yolo(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x_center = (box[0] + box[2]) / 2.0
    y_center = (box[1] + box[3]) / 2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    
    x_center_norm = x_center * dw
    y_center_norm = y_center * dh
    w_norm = w * dw
    h_norm = h * dh
    
    return (x_center_norm, y_center_norm, w_norm, h_norm)

def process_dataset_split(dataset_root, split, class_to_idx):
    img_dir = os.path.join(dataset_root, "images", split)
    xml_dir = os.path.join(dataset_root, f"{split}_annots")
    label_dir = os.path.join(dataset_root, "labels", split)
    
    os.makedirs(label_dir, exist_ok=True)

    if not (os.path.exists(xml_dir) and os.path.exists(img_dir)):
        return

    xml_files = glob.glob(os.path.join(xml_dir, "*.xml"))
    img_files_in_split = {os.path.basename(f) for f in glob.glob(os.path.join(img_dir, "*"))}
    
    for xml_path in xml_files:
        base = os.path.splitext(os.path.basename(xml_path))[0]
        label_path = os.path.join(label_dir, f"{base}.txt")
        yolo_lines = []
        
        try:
            root = ET.parse(xml_path).getroot()
            objects_found = [obj for obj in root.findall('object') if obj.findtext('name') in class_to_idx]
            
            if not objects_found:
                with open(label_path, 'w') as f: f.write('')
                continue

            size_elem = root.find('size')
            img_w, img_h = 0, 0
            
            if size_elem is not None:
                img_w = int(size_elem.findtext('width'))
                img_h = int(size_elem.findtext('height'))
            else:
                img_path = None
                img_found = False
                for ext in ['.png','.jpg','.jpeg','.tif', '.tiff']:
                    potential_file = base + ext
                    if potential_file in img_files_in_split:
                        img_path = os.path.join(img_dir, potential_file)
                        img_found = True
                        break
                if not img_found: continue
                with Image.open(img_path) as img:
                    img_w, img_h = img.size
            
            for obj in objects_found:
                class_id = class_to_idx[obj.findtext('name')]
                bnd = obj.find('bndbox')
                if bnd is None: continue
                
                xmin = int(float(bnd.findtext('xmin')))
                ymin = int(float(bnd.findtext('ymin')))
                xmax = int(float(bnd.findtext('xmax')))
                ymax = int(float(bnd.findtext('ymax')))
                
                yolo_bbox = convert_bbox_to_yolo((img_w, img_h), (xmin, ymin, xmax, ymax))
                yolo_lines.append(f"{class_id} {' '.join(map(str, yolo_bbox))}")
            
            with open(label_path, 'w') as f: f.write('\n'.join(yolo_lines))
                
        except Exception as e:
            print(f"Error processing {xml_path}: {e}")

def prepare_dataset(dataset_root, dataset_name, classes_to_train):
    class_to_idx = {name: i for i, name in enumerate(classes_to_train)}
    
    yaml_data = {
        'path': dataset_root,
        'train': os.path.join('images', 'train'),
        'val': os.path.join('images', 'valid'),
        'test': os.path.join('images', 'test'),
        'names': classes_to_train
    }
    
    yaml_path = os.path.join(dataset_root, f'{dataset_name.replace(" ", "_").lower()}_dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_data, f, sort_keys=False, default_flow_style=False)
    
    for split in ["train", "valid", "test"]:
        process_dataset_split(dataset_root, split, class_to_idx)
        
    return yaml_path

DATASET_NAME = "UAV 1"
BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "Datasets"))
CLASSES_TO_TRAIN = ['building']
RUN_NAME = f'yolo11n_{DATASET_NAME.replace(" ", "_").lower()}_building_only_WITH_aug_clean'

EPOCHS = 100
BATCH_SIZE = 24
WORKERS = 12
IMGSZ = 640

dataset_root = os.path.join(BASE_PATH, DATASET_NAME)
yaml_config_path = os.path.join(dataset_root, f'{DATASET_NAME.replace(" ", "_").lower()}_dataset.yaml')
train_labels_path = os.path.join(dataset_root, "labels", "train")

if not os.path.exists(dataset_root):
    print(f"Error: Dataset root not found at {dataset_root}")
else:
    if not (os.path.exists(yaml_config_path) and os.path.exists(train_labels_path)):
        prepare_dataset(dataset_root, DATASET_NAME, CLASSES_TO_TRAIN)
    
    model = YOLO('yolo11n.pt') 
    try:
        model.train(
            data=yaml_config_path,
            epochs=EPOCHS,
            imgsz=IMGSZ,
            batch=BATCH_SIZE,
            workers=WORKERS,
            name=RUN_NAME,
            verbose=False
        )

        best_model_path = os.path.join('runs', 'detect', RUN_NAME, 'weights', 'best.pt')
        if os.path.exists(best_model_path):
            model = YOLO(best_model_path)
            model.val(split='test', verbose=False)
        else:
            print(f"Validation failed: Could not find best model at {best_model_path}")
    
    except Exception as e:
        print(f"\nAn error occurred during training: {e}")


New https://pypi.org/project/ultralytics/8.3.223 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.40  Python-3.11.9 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=c:\Users\Think\Desktop\NesvarbuSefoSpecialus\Datasets\UAV 1\uav_1_dataset.yaml, epochs=100, time=None, patience=100, batch=24, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=12, project=None, name=yolo11n_uav_1_building_only_WITH_aug_clean, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, aug

train: Scanning C:\Users\Think\Desktop\NesvarbuSefoSpecialus\Datasets\UAV 1\labels\train.cache... 717 images, 181 backgrounds, 0 corrupt: 100%|██████████| 717/717 [00:00<?, ?it/s]
val: Scanning C:\Users\Think\Desktop\NesvarbuSefoSpecialus\Datasets\UAV 1\labels\valid.cache... 84 images, 21 backgrounds, 0 corrupt: 100%|██████████| 84/84 [00:00<?, ?it/s]


Plotting labels to runs\detect\yolo11n_uav_1_building_only_WITH_aug_clean\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005625000000000001), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 12 dataloader workers
Logging results to runs\detect\yolo11n_uav_1_building_only_WITH_aug_clean
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      6.05G      2.316      2.852      1.241        378        640: 100%|██████████| 30/30 [00:06<00:00,  4.61it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.38it/s]

                   all         84       1391      0.018      0.326     0.0407     0.0139


...


100 epochs completed in 0.185 hours.
Optimizer stripped from runs\detect\yolo11n_uav_1_building_only_WITH_aug_clean\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\yolo11n_uav_1_building_only_WITH_aug_clean\weights\best.pt, 5.5MB

Validating runs\detect\yolo11n_uav_1_building_only_WITH_aug_clean\weights\best.pt...
Ultralytics 8.3.40  Python-3.11.9 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.99it/s]


                   all         84       1391      0.791      0.733      0.799      0.464
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\yolo11n_uav_1_building_only_WITH_aug_clean
Ultralytics 8.3.40  Python-3.11.9 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients


val: Scanning C:\Users\Think\Desktop\NesvarbuSefoSpecialus\Datasets\UAV 1\labels\test.cache... 43 images, 11 backgrounds, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:04<00:00,  1.62s/it]


                   all         43        630      0.728      0.692      0.726      0.396
Speed: 2.0ms preprocess, 9.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\val3


In [ ]:
import os
import glob
import xml.etree.ElementTree as ET
import yaml
from PIL import Image
from ultralytics import YOLO

def convert_bbox_to_yolo(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x_center = (box[0] + box[2]) / 2.0
    y_center = (box[1] + box[3]) / 2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    
    x_center_norm = x_center * dw
    y_center_norm = y_center * dh
    w_norm = w * dw
    h_norm = h * dh
    
    return (x_center_norm, y_center_norm, w_norm, h_norm)

def process_dataset_split(dataset_root, split, class_to_idx):
    img_dir = os.path.join(dataset_root, "images", split)
    xml_dir = os.path.join(dataset_root, f"{split}_annots")
    label_dir = os.path.join(dataset_root, "labels", split)
    
    os.makedirs(label_dir, exist_ok=True)

    if not (os.path.exists(xml_dir) and os.path.exists(img_dir)):
        return

    xml_files = glob.glob(os.path.join(xml_dir, "*.xml"))
    img_files_in_split = {os.path.basename(f) for f in glob.glob(os.path.join(img_dir, "*"))}
    
    for xml_path in xml_files:
        base = os.path.splitext(os.path.basename(xml_path))[0]
        label_path = os.path.join(label_dir, f"{base}.txt")
        yolo_lines = []
        
        try:
            root = ET.parse(xml_path).getroot()
            objects_found = [obj for obj in root.findall('object') if obj.findtext('name') in class_to_idx]
            
            if not objects_found:
                with open(label_path, 'w') as f: f.write('')
                continue

            size_elem = root.find('size')
            img_w, img_h = 0, 0
            
            if size_elem is not None:
                img_w = int(size_elem.findtext('width'))
                img_h = int(size_elem.findtext('height'))
            else:
                img_path = None
                img_found = False
                for ext in ['.png','.jpg','.jpeg','.tif', '.tiff']:
                    potential_file = base + ext
                    if potential_file in img_files_in_split:
                        img_path = os.path.join(img_dir, potential_file)
                        img_found = True
                        break
                if not img_found: continue
                with Image.open(img_path) as img:
                    img_w, img_h = img.size
            
            for obj in objects_found:
                class_id = class_to_idx[obj.findtext('name')]
                bnd = obj.find('bndbox')
                if bnd is None: continue
                
                xmin = int(float(bnd.findtext('xmin')))
                ymin = int(float(bnd.findtext('ymin')))
                xmax = int(float(bnd.findtext('xmax')))
                ymax = int(float(bnd.findtext('ymax')))
                
                yolo_bbox = convert_bbox_to_yolo((img_w, img_h), (xmin, ymin, xmax, ymax))
                yolo_lines.append(f"{class_id} {' '.join(map(str, yolo_bbox))}")
            
            with open(label_path, 'w') as f: f.write('\n'.join(yolo_lines))
                
        except Exception as e:
            print(f"Error processing {xml_path}: {e}")

def prepare_dataset(dataset_root, dataset_name, classes_to_train):
    class_to_idx = {name: i for i, name in enumerate(classes_to_train)}
    
    yaml_data = {
        'path': dataset_root,
        'train': os.path.join('images', 'train'),
        'val': os.path.join('images', 'valid'),
        'test': os.path.join('images', 'test'),
        'names': classes_to_train
    }
    
    yaml_path = os.path.join(dataset_root, f'{dataset_name.replace(" ", "_").lower()}_dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_data, f, sort_keys=False, default_flow_style=False)
    
    for split in ["train", "valid", "test"]:
        process_dataset_split(dataset_root, split, class_to_idx)
        
    return yaml_path

DATASET_NAME = "UAV 1"
BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "Datasets"))
CLASSES_TO_TRAIN = ['building']
RUN_NAME = f'yolo11n_{DATASET_NAME.replace(" ", "_").lower()}_building_only_HEAVY_aug'

EPOCHS = 100
BATCH_SIZE = 24
WORKERS = 12
IMGSZ = 640

dataset_root = os.path.join(BASE_PATH, DATASET_NAME)
yaml_config_path = os.path.join(dataset_root, f'{DATASET_NAME.replace(" ", "_").lower()}_dataset.yaml')
train_labels_path = os.path.join(dataset_root, "labels", "train")

if not os.path.exists(dataset_root):
    print(f"Error: Dataset root not found at {dataset_root}")
else:
    if not (os.path.exists(yaml_config_path) and os.path.exists(train_labels_path)):
        prepare_dataset(dataset_root, DATASET_NAME, CLASSES_TO_TRAIN)
    
    model = YOLO('yolo11n.pt') 
    try:
        model.train(
            data=yaml_config_path,
            epochs=EPOCHS,
            imgsz=IMGSZ,
            batch=BATCH_SIZE,
            workers=WORKERS,
            name=RUN_NAME,
            verbose=False,
            
            hsv_h=0.015,
            hsv_s=0.7,
            hsv_v=0.4,
            degrees=90.0, # increased from 0.0
            translate=0.1,
            scale=0.5,
            shear=10.0, # increased from 0.0
            perspective=0.001, # increased from 0.0
            flipud=0.5, # increased from 0.0
            fliplr=0.5,
            bgr=0.0,
            mosaic=1.0,
            mixup=0.1, # increased from 0.0
            copy_paste=0.0
        )

        best_model_path = os.path.join('runs', 'detect', RUN_NAME, 'weights', 'best.pt')
        if os.path.exists(best_model_path):
            model = YOLO(best_model_path)
            model.val(split='test', verbose=False)
        else:
            print(f"Validation failed: Could not find best model at {best_model_path}")
    
    except Exception as e:
        print(f"\nAn error occurred during training: {e}")


New https://pypi.org/project/ultralytics/8.3.223 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.40  Python-3.11.9 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=c:\Users\Think\Desktop\NesvarbuSefoSpecialus\Datasets\UAV 1\uav_1_dataset.yaml, epochs=100, time=None, patience=100, batch=24, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=12, project=None, name=yolo11n_uav_1_building_only_HEAVY_aug, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=

train: Scanning C:\Users\Think\Desktop\NesvarbuSefoSpecialus\Datasets\UAV 1\labels\train.cache... 717 images, 181 backgrounds, 0 corrupt: 100%|██████████| 717/717 [00:00<?, ?it/s]
val: Scanning C:\Users\Think\Desktop\NesvarbuSefoSpecialus\Datasets\UAV 1\labels\valid.cache... 84 images, 21 backgrounds, 0 corrupt: 100%|██████████| 84/84 [00:00<?, ?it/s]


Plotting labels to runs\detect\yolo11n_uav_1_building_only_HEAVY_aug\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005625000000000001), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 12 dataloader workers
Logging results to runs\detect\yolo11n_uav_1_building_only_HEAVY_aug
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      6.07G      2.547      2.932       1.47        751        640: 100%|██████████| 30/30 [00:06<00:00,  4.63it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.85it/s]

                   all         84       1391     0.0154       0.28     0.0344    0.00894


...


100 epochs completed in 0.187 hours.
Optimizer stripped from runs\detect\yolo11n_uav_1_building_only_HEAVY_aug\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\yolo11n_uav_1_building_only_HEAVY_aug\weights\best.pt, 5.5MB

Validating runs\detect\yolo11n_uav_1_building_only_HEAVY_aug\weights\best.pt...
Ultralytics 8.3.40  Python-3.11.9 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.90it/s]


                   all         84       1391      0.778      0.689      0.766      0.391
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\yolo11n_uav_1_building_only_HEAVY_aug
Ultralytics 8.3.40  Python-3.11.9 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients


val: Scanning C:\Users\Think\Desktop\NesvarbuSefoSpecialus\Datasets\UAV 1\labels\test.cache... 43 images, 11 backgrounds, 0 corrupt: 100%|██████████| 43/43 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:04<00:00,  1.39s/it]


                   all         43        630      0.727      0.684      0.741      0.365
Speed: 1.0ms preprocess, 10.9ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\val4


In [4]:
import os
import glob
from ultralytics import YOLO
from tqdm import tqdm

DATASET_NAME = "UAV 1"
BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "Datasets"))

RUN_NAME = "yolo11n_uav_1_building_only_HEAVY_aug" 
PREDICTION_FOLDER_NAME = "predictions_building_HEAVY_aug"

CONF_THRESHOLD = 0.25

weights_path = os.path.join('runs', 'detect', RUN_NAME, 'weights', 'best.pt')
source_dir = os.path.join(BASE_PATH, DATASET_NAME, "images", "test")
output_dir = os.path.join('runs', 'detect')

if not os.path.exists(weights_path):
    print(f"Error: Weights file not found at {weights_path}")
elif not os.path.exists(source_dir):
    print(f"Error: Test images directory not found at {source_dir}")
else:
    model = YOLO(weights_path)
    
    image_paths = []
    supported_exts = ['.png','.jpg','.jpeg','.tif', '.tiff']
    for ext in supported_exts:
        image_paths.extend(glob.glob(os.path.join(source_dir, f"*{ext}")))
        
    if not image_paths:
        print(f"No images found in {source_dir}")
    else:
        for img_path in tqdm(image_paths, desc=f"Running predictions with {RUN_NAME}"):
            model.predict(
                source=img_path,
                save=True,
                project=output_dir,
                name=PREDICTION_FOLDER_NAME,
                exist_ok=True,
                conf=CONF_THRESHOLD,
                save_txt=True,
                verbose=False
            )
        
        final_output_path = os.path.abspath(os.path.join(output_dir, PREDICTION_FOLDER_NAME))
        print(f"Prediction complete. Results saved to: {final_output_path}")


Running predictions with yolo11n_uav_1_building_only_HEAVY_aug:   0%|          | 0/43 [00:00<?, ?it/s]

Results saved to runs\detect\predictions_building_HEAVY_aug
1 label saved to runs\detect\predictions_building_HEAVY_aug\labels


Running predictions with yolo11n_uav_1_building_only_HEAVY_aug:   2%|▏         | 1/43 [00:00<00:19,  2.14it/s]

Results saved to runs\detect\predictions_building_HEAVY_aug
1 label saved to runs\detect\predictions_building_HEAVY_aug\labels
Results saved to runs\detect\predictions_building_HEAVY_aug
1 label saved to runs\detect\predictions_building_HEAVY_aug\labels


Running predictions with yolo11n_uav_1_building_only_HEAVY_aug:   7%|▋         | 3/43 [00:00<00:06,  6.27it/s]

...

Results saved to runs\detect\predictions_building_HEAVY_aug
28 labels saved to runs\detect\predictions_building_HEAVY_aug\labels
Results saved to runs\detect\predictions_building_HEAVY_aug
28 labels saved to runs\detect\predictions_building_HEAVY_aug\labels
Results saved to runs\detect\predictions_building_HEAVY_aug
29 labels saved to runs\detect\predictions_building_HEAVY_aug\labels


Running predictions with yolo11n_uav_1_building_only_HEAVY_aug:  93%|█████████▎| 40/43 [00:02<00:00, 18.62it/s]

Results saved to runs\detect\predictions_building_HEAVY_aug
30 labels saved to runs\detect\predictions_building_HEAVY_aug\labels
Results saved to runs\detect\predictions_building_HEAVY_aug
30 labels saved to runs\detect\predictions_building_HEAVY_aug\labels
Results saved to runs\detect\predictions_building_HEAVY_aug
30 labels saved to runs\detect\predictions_building_HEAVY_aug\labels


Running predictions with yolo11n_uav_1_building_only_HEAVY_aug: 100%|██████████| 43/43 [00:03<00:00, 14.02it/s]

Prediction complete. Results saved to: c:\Users\Think\Desktop\NesvarbuSefoSpecialus\App\runs\detect\predictions_building_HEAVY_aug
